In [ ]:
#Task: Schedule a meeting for a specific group of Theology professors 
#Meeting must take place at a time when none of the group is teaching a course

#Information available: downloadable csv file with course information for all Theology courses
#This includes professor names and course meeting days and times

#Goal of this program: select data for courses taught by the desired group of professors
#and reformat it into an easy-to-read weekly conglomerate schedule
#so it is quick and easy to see the times when none of the group is teaching a course

In [1]:
#import libraries
import pandas as pd
import numpy as np

In [2]:
#read in the file (find data on Notre Dame Class Search; Download to Excel; Save as CSV File)
df = pd.read_csv('ClassSearchSp21DataSheet.csv',encoding = 'unicode_escape')

In [3]:
#check data
df.head()

,Course - Sec,Title,Cr,St,Max,Opn,Xlst,CRN,Instructor,When,Begin,End,Where
0,THEO10001-01,Found of Theo: Biblcl Historcl,3,OP,100,100,NaN,22127,"Anderson, Gary",M W - 10:25A - 11:15A,2/3/21,5/11/21,DeBartolo Performing Arts Ctr. 124
1,THEO10001-02,Found of Theo: Biblcl Historcl,3,OP,45,45,NaN,20392,"Dolcich-Ashley, Anselma",T R - 8:00A - 9:15A,2/3/21,5/11/21,Bond Hall 220
2,THEO10001-03,Found of Theo: Biblcl Historcl,3,OP,65,65,NaN,23859,"Reynolds, Gabriel",T R - 9:35A - 10:50A,2/3/21,5/11/21,Hesburgh Library 107
3,THEO10001-04,Found of Theo: Biblcl Historcl,3,OP,45,45,NaN,20508,"Radle, Gabriel",T R - 9:35A - 10:50A,2/3/21,5/11/21,ONLINE COURSE
4,THEO10001-05,Found of Theo: Biblcl Historcl,3,OP,45,45,NaN,20631,"Radle, Gabriel",T R - 11:10A - 12:25P,2/3/21,5/11/21,ONLINE COURSE


In [4]:
#drop unneeded columns
class_days_times = df.drop(['Cr', 'St', 'Max', 'Opn', 'Xlst', 'CRN', 'End'],axis=1)

In [5]:
#drop rows with January-term courses
class_days_times = class_days_times[(class_days_times['Begin']=='2/3/21')]

In [6]:
#drop remaining unneeded column
class_days_times = class_days_times.drop(['Begin'],axis=1)

In [7]:
class_days_times.head()

,Course - Sec,Title,Instructor,When,Where
0,THEO10001-01,Found of Theo: Biblcl Historcl,"Anderson, Gary",M W - 10:25A - 11:15A,DeBartolo Performing Arts Ctr. 124
1,THEO10001-02,Found of Theo: Biblcl Historcl,"Dolcich-Ashley, Anselma",T R - 8:00A - 9:15A,Bond Hall 220
2,THEO10001-03,Found of Theo: Biblcl Historcl,"Reynolds, Gabriel",T R - 9:35A - 10:50A,Hesburgh Library 107
3,THEO10001-04,Found of Theo: Biblcl Historcl,"Radle, Gabriel",T R - 9:35A - 10:50A,ONLINE COURSE
4,THEO10001-05,Found of Theo: Biblcl Historcl,"Radle, Gabriel",T R - 11:10A - 12:25P,ONLINE COURSE


In [8]:
#drop rows in which "when" has multiple sections (these rows begin with parentheses)
class_days_times['DropRowsWithParens']=class_days_times['When'].str.get(0)
class_days_times = class_days_times[(class_days_times['DropRowsWithParens']!='(')]
class_days_times = class_days_times.drop(['DropRowsWithParens'],axis=1)

#a better solution, which I have yet to implement:
#identify rows with parentheses
#divide the cell at parentheses
#create multiple rows in sheet; copy all other info down, but divide sections in the "when" column

In [9]:
#drop rows where time is TBA (often directed readings, etc.)
class_days_times = class_days_times[(class_days_times['When']!='TBA')]

In [10]:
#create new columns for start and end time and for meeting days (by dividing and then removing "When" column)
class_days_times['When'] = class_days_times['When'].apply(lambda x: x.split('-'))
class_days_times['MeetingDays'] = class_days_times['When'].apply(lambda x: x[0])
class_days_times['StartTime'] = class_days_times['When'].apply(lambda x: x[1])
class_days_times['EndTime'] = class_days_times['When'].apply(lambda x: x[2])
class_days_times = class_days_times[class_days_times['MeetingDays']!='(01) M '] #no longer needed, 
                                                    #since we removed rows with parentheses?
class_days_times = class_days_times.drop(['When'],axis=1)

In [11]:
#checking to make sure the column division was successful
class_days_times.head()

,Course - Sec,Title,Instructor,Where,MeetingDays,StartTime,EndTime
0,THEO10001-01,Found of Theo: Biblcl Historcl,"Anderson, Gary",DeBartolo Performing Arts Ctr. 124,M W,10:25A,11:15A
1,THEO10001-02,Found of Theo: Biblcl Historcl,"Dolcich-Ashley, Anselma",Bond Hall 220,T R,8:00A,9:15A
2,THEO10001-03,Found of Theo: Biblcl Historcl,"Reynolds, Gabriel",Hesburgh Library 107,T R,9:35A,10:50A
3,THEO10001-04,Found of Theo: Biblcl Historcl,"Radle, Gabriel",ONLINE COURSE,T R,9:35A,10:50A
4,THEO10001-05,Found of Theo: Biblcl Historcl,"Radle, Gabriel",ONLINE COURSE,T R,11:10A,12:25P


In [12]:
#function to be used in creating columns for days of the week
def d_in_string(d,string):
    for c in string:
        if c==d:
            return 1
    return 0

In [13]:
#create columns for days of the week
class_days_times['Monday'] = class_days_times['MeetingDays'].apply(lambda string: d_in_string('M',string))
class_days_times['Tuesday'] = class_days_times['MeetingDays'].apply(lambda string: d_in_string('T',string))
class_days_times['Wednesday'] = class_days_times['MeetingDays'].apply(lambda string: d_in_string('W',string))
class_days_times['Thursday'] = class_days_times['MeetingDays'].apply(lambda string: d_in_string('R',string))
class_days_times['Friday'] = class_days_times['MeetingDays'].apply(lambda string: d_in_string('F',string))
class_days_times['Uday'] = class_days_times['MeetingDays'].apply(lambda string: d_in_string('U',string))

In [14]:
#function to convert times to integers in order to sort them according to start time (military time)
def time_to_int(string):
    string = string.replace(':','')
    string = string.replace(' ','')
    if string[-1]=='P':
        output = int(string[:-1])+1200
        if output>2400:
            output=output-1200
    elif string[-1]=='A':
        output = int(string[:-1])
    else:
        output = Error
    return output

In [15]:
#use function above to convert times to integers
class_days_times['StartTime'] = class_days_times['StartTime'].apply(lambda x: time_to_int(x))
class_days_times['EndTime'] = class_days_times['EndTime'].apply(lambda x: time_to_int(x))

In [16]:
#checking that time to integer conversion was successful
class_days_times.head()

,Course - Sec,Title,Instructor,Where,MeetingDays,StartTime,EndTime,Monday,Tuesday,Wednesday,Thursday,Friday,Uday
0,THEO10001-01,Found of Theo: Biblcl Historcl,"Anderson, Gary",DeBartolo Performing Arts Ctr. 124,M W,1025,1115,1,0,1,0,0,0
1,THEO10001-02,Found of Theo: Biblcl Historcl,"Dolcich-Ashley, Anselma",Bond Hall 220,T R,800,915,0,1,0,1,0,0
2,THEO10001-03,Found of Theo: Biblcl Historcl,"Reynolds, Gabriel",Hesburgh Library 107,T R,935,1050,0,1,0,1,0,0
3,THEO10001-04,Found of Theo: Biblcl Historcl,"Radle, Gabriel",ONLINE COURSE,T R,935,1050,0,1,0,1,0,0
4,THEO10001-05,Found of Theo: Biblcl Historcl,"Radle, Gabriel",ONLINE COURSE,T R,1110,1225,0,1,0,1,0,0


In [17]:
#reference tool for finding instructor
#the name must be typed exactly as shown here in order to retrieve an instructor's course schedule
class_days_times['Instructor'].unique()

array(['Anderson, Gary', 'Dolcich-Ashley, Anselma', 'Reynolds, Gabriel',
       'Radle, Gabriel', 'Cavadini, Catherine', 'Baglow, Christopher',
       'Oakes, Kenneth', 'De La Noval, Roberto', 'Martin, James',
       'Battin, Steven', 'Roumas, Sarah', 'Lacy, Travis', 'McLean, John',
       'Johnson, Sarah', 'TBA', 'Edwards, Robert', "O'Malley, Timothy",
       'Novick, Michael', 'Belcher, Kimberly', 'Matovina, Timothy',
       'Malkovsky, Bradley', 'Porter, Jean', 'Anatolios, Khaled',
       'Arner, Neil', 'Torrance, Alexis', 'Page, Hugh', 'Brown, Jeremy',
       'Gerdon, Ian', 'Astell, Ann', 'Avvakumov, Yury', 'Cavadini, John',
       'Wawrykow, Joseph', 'Pagliarini, Anthony', 'Johnson, Maxwell',
       'Ehrman, Terrence', 'Clairmont, David', 'Akesseh, Ebenezer',
       'Pfeil, Margaret', 'Savides, Steven', 'Lantigua, David',
       'Idoko, Patricia', "Sirry, Mun'im", 'Pomplun, Robert',
       'Betz, John', 'DeLorenzo, Leonard', 'Kollman, Paul',
       'Hemphill, Hannah', 'Kiley, Robe

In [26]:
#create data frame with course schedule info for only the instructors in question

#enter the instructors' names exactly as they are listed in the field above
special = class_days_times[(class_days_times['Instructor']=='Matovina, Timothy')
                 |(class_days_times['Instructor']=='Cavadini, John')
                 |(class_days_times['Instructor']=='Cavadini, Katherine')
                 |(class_days_times['Instructor']=='Jensen, Robin')
                 |(class_days_times['Instructor']=='Anatolios, Khaled')
                 |(class_days_times['Instructor']=='Reynolds, Gabriel')
                 |(class_days_times['Instructor']=='Novick, Michael')
                 |(class_days_times['Instructor']=='Johnson, Maxwell')
                 |(class_days_times['Instructor']=='Ashley, James')]

#ideally I would like to change this step to be in some kind of checkbox menu
#or at least would account for spelling errors 
#and courses listed with both faculty member and graduate student as instructors

In [27]:
#lists all courses taught by professors in question
special

,Course - Sec,Title,Instructor,Where,MeetingDays,StartTime,EndTime,Monday,Tuesday,Wednesday,Thursday,Friday,Uday
2,THEO10001-03,Found of Theo: Biblcl Historcl,"Reynolds, Gabriel",Hesburgh Library 107,T R,935,1050,0,1,0,1,0,0
21,THEO10002-03,Found of Theo: Biblcl Historcl,"Novick, Michael",ONLINE COURSE,M W,1245,1400,1,0,1,0,0,0
22,THEO10002-04,Found of Theo: Biblcl Historcl,"Novick, Michael",ONLINE COURSE,M W,1420,1535,1,0,1,0,0,0
24,THEO10011-01,Foundations of Theology,"Matovina, Timothy",Jordan Hall of Science 101,T R,1420,1535,0,1,0,1,0,0
31,THEO13002-02,Hnrs Fdt of Theo: Biblcl/Hist,"Anatolios, Khaled",DeBartolo Hall 310,M W,1110,1225,1,0,1,0,0,0
68,THEO20251-01,The Catholic Faith,"Cavadini, John",Stepan Center 101,T R,1420,1535,0,1,0,1,0,0
69,THEO20251-02,The Catholic Faith,"Cavadini, John",Stepan Center 101,T R,1420,1535,0,1,0,1,0,0
70,THEO20251-03,The Catholic Faith,"Cavadini, John",Stepan Center 101,T R,1420,1535,0,1,0,1,0,0
71,THEO20251-04,The Catholic Faith,"Cavadini, John",Stepan Center 101,T R,1420,1535,0,1,0,1,0,0
72,THEO20251-05*,The Catholic Faith,"Cavadini, John",Stepan Center 101,T R,1420,1535,0,1,0,1,0,0


In [28]:
#example: show only the courses that meet on Tuesdays
special[special['Tuesday']==1].head()

,Course - Sec,Title,Instructor,Where,MeetingDays,StartTime,EndTime,Monday,Tuesday,Wednesday,Thursday,Friday,Uday
2,THEO10001-03,Found of Theo: Biblcl Historcl,"Reynolds, Gabriel",Hesburgh Library 107,T R,935,1050,0,1,0,1,0,0
24,THEO10011-01,Foundations of Theology,"Matovina, Timothy",Jordan Hall of Science 101,T R,1420,1535,0,1,0,1,0,0
68,THEO20251-01,The Catholic Faith,"Cavadini, John",Stepan Center 101,T R,1420,1535,0,1,0,1,0,0
69,THEO20251-02,The Catholic Faith,"Cavadini, John",Stepan Center 101,T R,1420,1535,0,1,0,1,0,0
70,THEO20251-03,The Catholic Faith,"Cavadini, John",Stepan Center 101,T R,1420,1535,0,1,0,1,0,0


In [29]:
#display course meeting times (number of courses that meet during each given timeslot)
special.groupby(by = ['StartTime','EndTime']).sum()

Monday  Tuesday  Wednesday  Thursday  Friday  Uday
StartTime EndTime                                                    
800       915           2        0          2         0       0     0
900       1000          0        0          0         0       1     0
935       1050          5        2          5         2       0     0
1110      1225          1        0          1         0       0     0
1245      1400          1        2          1         2       0     0
          1530          0        1          0         0       0     0
1420      1535          4        6          4         6       0     0
1555      1710          0        1          0         1       0     0
          1840          0        0          0         0       1     0
1630      1730          1        0          0         0       0     0
1740      1830          0        1          0         0       0     0

In [ ]:
#if any professor in the group teaches during a certain timeslot, 
#this layout shows the number of sections running in that timeslot
#it is helpful for identifying gaps in teaching schedules 
#in order to find a meeting time when a certain group of professors is available